# IMPRORT

In [6]:
import pandas as pd

In [6]:
pip install pyarrow

  Using cached pyarrow-14.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-14.0.1-cp310-cp310-manylinux_2_28_x86_64.whl (38.0 MB)

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
import numpy as np

input_file = 'post_processed_data.ftr'
df = pd.read_feather(input_file, columns=None, use_threads=True)
print(df.head(2))
print(df['pixels'][0].shape)
# remove the exteior [] of df pixels
#df['pixels'] = df['pixels'].apply(lambda x: x[0])
print(df['pixels'][:100])
print(df['pixels'].shape)
print(df['pixels'][0].shape)

for i, arr in enumerate(df['pixels'][0]):
    print(f"Array {i}: Shape - {arr.shape}")


numpy_array = np.array(df['pixels'][0])
print("Shape of the numpy...:", numpy_array.shape)
total_elements = numpy_array.size
print("Total number of elements:", total_elements)
reshaped_array = numpy_array.reshape(16, 73, 10)


  segment_id polygon_id class_id  \
0     367854        0.0      3.0   
1     367855        0.0      3.0   

                                              pixels  
0  [[0.6406995, 0.59669423, 0.63548833, 0.6320272...  
1  [[0.47058824, 0.47603306, 0.42916092, 0.465076...  
(16,)
0     [[0.6406995, 0.59669423, 0.63548833, 0.6320272...
1     [[0.47058824, 0.47603306, 0.42916092, 0.465076...
2     [[0.43561208, 0.36859503, 0.3645117, 0.4548552...
3     [[0.7837838, 0.75206614, 0.5983494, 0.6592845,...
4     [[0.5643879, 0.57520664, 0.4621733, 0.54003406...
                            ...                        
95    [[0.33863276, 0.37851238, 0.21320495, 0.369676...
96    [[0.52305245, 0.5008264, 0.5006878, 0.54344124...
97    [[0.5834658, 0.54380167, 0.54332876, 0.5877342...
98    [[0.29888713, 0.35041323, 0.28060523, 0.320272...
99    [[0.4308426, 0.4842975, 0.38651994, 0.47529814...
Name: pixels, Length: 100, dtype: object
(4795,)
(16,)
Array 0: Shape - (730,)
Array 1: Shape - (730,)
A

ValueError: cannot reshape array of size 16 into shape (16,73,10)

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(array[0])

ValueError: setting an array element with a sequence.

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split

df['pixels'] = df['pixels'].apply(lambda x: np.array(x))

print(df['pixels'].shape) #(4795,)
print(len(df['pixels'][0])) #730
print(df['pixels'][0].shape) #(730,)

print(df['pixels'][0])
print(df['pixels'].shape)
print(df['pixels'][1].shape)

(4795,)
16
(16,)
[array([0.6406995 , 0.59669423, 0.63548833, 0.63202727, 0.65957445,
        0.49824563, 0.54585636, 0.511041  , 0.52150536, 0.4963181 ,
        0.64      , 0.5751724 , 0.48269582, 0.46713287, 0.4301802 ,
        0.3372093 , 0.2776747 , 0.4253876 , 0.5020492 , 0.62374246,
        0.64394546, 0.65371025, 0.6244813 , 0.52614015, 0.4515464 ,
        0.3760603 , 0.34828246, 0.25601375, 0.31711555, 0.33090246,
        0.34794775, 0.32139304, 0.27167115, 0.26536313, 0.23888889,
        0.23178017, 0.21032746, 0.3249694 , 0.29416057, 0.20978503,
        0.14712153, 0.13824192, 0.1426392 , 0.14430015, 0.14174253,
        0.1417004 , 0.15678777, 0.15600243, 0.1569966 , 0.21550497,
        0.27700222, 0.25789472, 0.21925753, 0.29100528, 0.24820144,
        0.19376026, 0.19973545, 0.44612068, 0.4849624 , 0.49740034,
        0.42131978, 0.2447217 , 0.44700462, 0.5015015 , 0.5514905 ,
        0.61558783, 0.65776294, 0.61360717, 0.6841416 , 0.6170543 ,
        0.6369048 , 0.5371622 ,

In [73]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(df['pixels'])

ValueError: setting an array element with a sequence.

In [ ]:
# df['pixels']
# Splitting the data into train valid and test sets,   ['pixels'] for the features and ['class_id'] for the labels
X_train, X_test, y_train, y_test = train_test_split(df['pixels'], df['class_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
##print(X_train[0].shape, X_valid.shape, X_test.shape)
#print(X_train[0])

In [32]:
import numpy as np

def extract_sequences(column_data):
    extracted_data = []
    for item in column_data:
        sequences = np.array(item).reshape(-1, 73, 10)  # Reshape the sequences if they are of shape (16, 73, 10)
        extracted_data.append(sequences)
    return extracted_data

# Assuming df['pixels'] contains your data
extracted_sequences = extract_sequences(df['pixels'])

# Check the length of the extracted sequences and an individual element's shape
print('Length of Extracted Sequences:', len(extracted_sequences))
print('Individual Element Shape:', extracted_sequences[0].shape)
print(extracted_sequences[0])

X_train, X_test, y_train, y_test = train_test_split(extracted_sequences, df['class_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print (X_train[0].shape, X_valid[0].shape, X_test[0].shape)


Length of Extracted Sequences: 4795
Individual Element Shape: (16, 73, 10)
[[[array([0.6406995 , 0.59669423, 0.63548833, 0.63202727, 0.65957445,
          0.49824563, 0.54585636, 0.511041  , 0.52150536, 0.4963181 ,
          0.64      , 0.5751724 , 0.48269582, 0.46713287, 0.4301802 ,
          0.3372093 , 0.2776747 , 0.4253876 , 0.5020492 , 0.62374246,
          0.64394546, 0.65371025, 0.6244813 , 0.52614015, 0.4515464 ,
          0.3760603 , 0.34828246, 0.25601375, 0.31711555, 0.33090246,
          0.34794775, 0.32139304, 0.27167115, 0.26536313, 0.23888889,
          0.23178017, 0.21032746, 0.3249694 , 0.29416057, 0.20978503,
          0.14712153, 0.13824192, 0.1426392 , 0.14430015, 0.14174253,
          0.1417004 , 0.15678777, 0.15600243, 0.1569966 , 0.21550497,
          0.27700222, 0.25789472, 0.21925753, 0.29100528, 0.24820144,
          0.19376026, 0.19973545, 0.44612068, 0.4849624 , 0.49740034,
          0.42131978, 0.2447217 , 0.44700462, 0.5015015 , 0.5514905 ,
          0.615

In [3]:
from typing import Any
from py3.k_means import PixelValueGenerator, CustomKMeans
import numpy as np
from py3.CNN_model import Cplx_CustomCNN_1D
from py3.Attention_Layer import CustomAttentionLayer
from py3.classification import CustomClassifierModel
import tensorflow as tf
 
class MODEL():
    def __init__(self, num_classes=7, num_clusters=2):
        self.num_classes = num_classes
        self.num_clusters = num_clusters
        self.kmeans = CustomKMeans(n_clusters=self.num_clusters) 
        self.input_shape = self.calculate_input_shape()
        self.enc = Cplx_CustomCNN_1D(input_shape_custom=self.input_shape, num_classes=self.num_classes)
        self.attn = CustomAttentionLayer(units=64)
        self.classifier = CustomClassifierModel(num_classes=self.num_classes)
        
    def calculate_input_shape(self):
        self.kmeans.fit(input) # AAA CHANGER !!!! 
        cluster_centers = self.kmeans.get_cluster_centers()
        print("cluster_centers shape ",cluster_centers.shape)
        return (cluster_centers[0].reshape((1,) + cluster_centers[0].shape)).shape[1:]
    
    def build(self, learning_rate=0.001):
        self.model = tf.keras.models.Sequential([
            self.enc,
            self.attn,
            self.classifier
        ])
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])

    def train(self, train_data, train_labels, epochs=10, batch_size=32):
        self.model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)

2023-11-18 13:34:14.501595: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 13:34:16.969468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 13:34:16.969515: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 13:34:16.974046: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 13:34:17.983268: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 13:34:17.984064: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens